# Negative Sampling  : 네거티브 샘플링

### Embedding과 EmbeddingDot 계층

In [1]:
import numpy as np

# nn_layers.py에 추가하여 놓는다

# Embedding 계층
class Embedding :
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    # 순전파
    def forward(self,idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    # 역전파
    def backward(self, dout):  # 중복 인덱스가 있어도 올바르게 처리, 속도가 빠름
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)  
        return None       

In [2]:
# EmbeddingDot 계층
# nn_layers.py에 추가하여 놓는다

class EmbeddingDot:
    def __init__(self,W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self,h,idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W*h,axis=1)   # 1차원 출력
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0],1) # 2차원으로 변환
        
        dtarget_W = dout*h  # sum <--> repeat, 브로드캐스트
        self.embed.backward(dtarget_W)
        
        dh = dout*target_W  # 브로드캐스트
        return dh

In [7]:
# EmbeddingDot 클래스 테스트 
W = np.arange(21).reshape(7,3)
print('W:\n',W)

idx = np.array([0,3,1])
print('idx:\n',idx)

h = W[[0,1,2]]
print('h:\n',h)

embed_dot = EmbeddingDot(W)
out = embed_dot.forward(h,idx)
print('out:\n',out)

print('h:\n',embed_dot.cache[0])
print('target_W:\n',embed_dot.cache[1])

W:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
idx:
 [0 3 1]
h:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
out:
 [  5 122  86]
h:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
target_W:
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]]


In [4]:
# EmbeddingDot 계층의 forward 함수내에서 변수의 값 변화 정보
idx = np.array([0,3,1])
embed = Embedding(W)
target_W = embed.forward(idx) # W에서 임베딩 처리
print('target_W:\n',target_W)

h = W[[0,1,2]]
print('h:\n',h)

temp = target_W * h
print('temp:\n',temp)

out = np.sum(temp,axis=1)
print('out:\n',out)

target_W:
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]]
h:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
temp:
 [[ 0  1  4]
 [27 40 55]
 [18 28 40]]
out:
 [  5 122  86]


### 네거티브 샘플링 기법
 https://ddiri01.tistory.com/310

In [17]:
# 0에서 9까지의 숫자 중 하나를 무작위로 샘플링
np.random.choice(10)

9

In [21]:
# words에서만 하나만 무작위로 샘플링
words = ['you','say','goodbye','i','hello','.']
np.random.choice(words)

'say'

In [28]:
# 5개만 무작위로 샘플링 (중복 허용)
np.random.choice(words,size=5)

array(['goodbye', 'hello', 'you', 'hello', 'hello'], dtype='<U7')

In [34]:
# 5개만 무작위로 샘플링 (중복 금지)
np.random.choice(words,size=5,replace=False)

array(['.', 'say', 'i', 'goodbye', 'you'], dtype='<U7')

In [45]:
# 주어진 확률 분포에 따라 샘플링
p = [0.5,0.1,0.05,0.2,0.05,0.1]  # 합이 1.0
print(sum(p))

np.random.choice(words,p = p)

1.0000000000000002


'you'

In [54]:
# 네거티브 샘플링에서는 기본 확률 분포에 0.75를 제곱해준다
# 확률분포 값들에 모두 0.75제곱 처리 ==> 원래 확률이 낮은 단어를 버리지 않기 위해서
p = [0.7,0.29,0.01]
print(sum(p))
new_p = np.power(p,0.75)
print(new_p)
new_p /= np.sum(new_p) # 다시 전체 요소의 합으로나누어 각 요소의 합이 1.0이 나오도록 새로운 확률 분포를 만든다
print(new_p)
print(sum(new_p))

1.0
[0.76528558 0.39518322 0.03162278]
[0.64196878 0.33150408 0.02652714]
1.0


In [55]:
print(2**4, np.power(2,4) ) # 2^4

16 16


In [ ]:
# 3개의 긍정 unigram 타겟을 주면  부정 맥락 2개씩 3개 생성해주는 클래스  